In [4]:
#Adapted from Ben's preprocessing work pushed on 14 Oct, 2020

import pandas as pd
import string
from datetime import datetime
import unidecode
from word2number import w2n
import gensim

import warnings
warnings.filterwarnings("ignore")


In [5]:
comments = pd.read_csv('data/train-balanced-sarcasm.csv')

In [6]:
comments

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


In [9]:
comments.groupby('label').count()

,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
label,,,,,,,,,
0,505405,505413,505413,505413,505413,505413,505413,505413,505413
1,505368,505413,505413,505413,505413,505413,505413,505413,505413


In [ ]:
comments['created_utc'][0]

## Processing Techniques Applied to the Comments

In [7]:
def year_splitter(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    year = date.year
    
    return f'{year}'

def month_extractor(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    month = date.month
    
    return f'{month:02}' 

def day_extractor(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    day= date.day
    
    return f'{day:02}' 

def hour_extractor(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    hour = date.hour
    
    return f'{hour:02}' 

def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    
    return text

def convert_numbers(text):
    token = text.split()

    final = ''
    for word in range(len(token)):
        try:
            number = w2n.word_to_num(token[word])
            final += str(number)
        except:
            final += token[word]
    
        if (word < len(token)):
            final += ' '
        
    return final

def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    
    return text

def remove_single_word(text):
    token = text.split()
    final = ''
    for word in range(len(token)):
        
        if len(token[word])>1:
            final += token[word]
            
            if (word < len(token)):
                final += ' '
    return final
    
def change_negation_words(text):
    
    negation = [ 'ain', 'aren', "arent", 'couldn',  "couldnt", 'didn', "didnt", 'doesn', "doesnt", "don", "dont", 'hadn', 
                "hadnt", 'hasn', "hasnt", 'haven', "havent", 'isn',  "isnt", 'ma', 'mightn', "mightnt", 'mustn', "mustnt", 
                'needn', "neednt", 'shan', "shant", 'shouldn', "shouldnt", 'wasn', "wasnt", 'weren', "werent", 'won', "wont",
                'wouldn', "wouldnt", "cannot"]
    
    token = text.split()
    final = ''
    for word in range(len(token)):
        if (token[word] not in negation):
            final += token[word]
            
            if (word < len(token)):
                final += ' '
        else:
            final += "not "
    return final  

# removed words: 'not', 'no'
def remove_stopwords(text):
    stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "youre", "youve", "youll", "youd", 'your', 
        'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "shes", 'her', 'hers', 'herself', 'it', 
        'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
        'that', "thatll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 
        'having', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
        'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
        'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
        'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
        'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 
        "dont", 'should', "shouldve", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', "theyre"]
    
    token = text.split()
    final = ''
    accept_word_count = 0
    
    for word in range(len(token)):
        if (token[word] not in stop):
            if (accept_word_count!=0):
                final += ' '
            final += token[word]
            accept_word_count += 1
            
    
    return final   

In [8]:
# removing the columns not really needed
comments.drop(['author', 'date'], axis = 1, inplace = True)
comments = comments.dropna()

In [9]:
# extraving the year, month, day and hour that the comment was made. May be useful in the future
comments['year'] = comments['created_utc'].map(lambda x: year_splitter(x))
comments['month'] = comments['created_utc'].map(lambda x: month_extractor(x))
comments['day'] = comments['created_utc'].map(lambda x: day_extractor(x))
comments['hour'] = comments['created_utc'].map(lambda x: hour_extractor(x))

In [10]:
comments

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,10,16,23
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,11,01,00
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,2016,09,22,21
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,2016,10,18,21
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,2016,12,30,17
...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",2009,04,25,00
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,2009,05,14,22
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,2009,01,11,00
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,2009,01,23,21


## Actual Comments

In [11]:
# removing all forms of punctuation, maintains spaces
comments['cleaned comment'] = comments['comment'].str.replace('[{}]'.format(string.punctuation), '')

# removing all upper case letters from each comment
comments['cleaned comment'] = comments['cleaned comment'].str.lower()

# converting accented characters to be standard
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_accented_chars)

# convert numbers to numeric where possible
comments['cleaned comment'] = comments['cleaned comment'].apply(convert_numbers)

# remove numbers from each comment
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_numbers)

# remove the tokens with single word
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_single_word)

# change the negation words from each comment
comments['cleaned comment'] = comments['cleaned comment'].apply(change_negation_words)


# removing stopwords from each comment. Stop words taken from NLTK's default stop word list
# removed words: 'not', 'no'
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_stopwords)

# removing any empty rows just incase a few comments were removed all together
comments = comments.dropna()

In [12]:
comments

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour,cleaned comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,10,16,23,nc nh
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,11,01,00,do know west teams play west teams east teams ...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,2016,09,22,21,underdogs earlier today since gronks announcem...
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,2016,10,18,21,meme not funny none new york nigga ones
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,2016,12,30,17,could use tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",2009,04,25,00,im sure iran korea technology create pigbirdhu...
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,2009,05,14,22,whatever do not vote green
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,2009,01,11,00,perhaps atheist conspiracy make christians loo...
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,2009,01,23,21,slavs got country called kosovo


## Parent comments

In [13]:
# doing the same for the parent comments
# removing punctuation
comments['cleaned parent comment'] = comments['parent_comment'].str.replace('[{}]'.format(string.punctuation), '')

# removing all upper case letters from each comment
comments['cleaned parent comment'] = comments['cleaned parent comment'].str.lower()

# converting accented characters to be standard
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_accented_chars)

# convert numbers to numeric where possible
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(convert_numbers)

# remove numbers from each comment
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_numbers)

# remove the tokens with single word
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_single_word)

# change the negation words from each comment
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(change_negation_words)

# removing stopwords from each comment. Stop words taken from NLTK's default stop word list
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_stopwords)

# removing any empty rows just incase a few comments were removed all together
comments = comments.dropna()

In [14]:
comments[(comments["cleaned comment"]=="")]

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour,cleaned comment,cleaned parent comment
25,0,466,Overwatch,5,-1,-1,2016-11-03 00:20:04,467 A lovely way!,2016,11,03,00,,lovely way
42,0,1571049,counting,1,-1,-1,2016-12-17 06:56:28,"1,571,048",2016,12,17,06,,
95,0,"1,512,798",counting,1,-1,-1,2016-11-18 20:26:43,"1,512,797",2016,11,18,20,,
118,0,L,EnoughTrumpSpam,5,-1,-1,2016-11-08 21:06:01,I,2016,11,08,21,,
225,0,*their,ClashRoyale,1,-1,-1,2016-11-01 13:27:16,Seriously all youtubers should combine to give...,2016,11,01,13,,seriously youtubers combine give nova taste me...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009592,0,This,politics,1,1,0,2010-02-13 20:55:29,That would leave about 15 people,2010,02,13,20,,would leave people
1009921,0,and?,WTF,0,0,0,2010-11-02 07:40:48,brb showing this to my voice and speech teache...,2010,11,02,07,,brb showing voice speech teacher college
1010514,1,This.,self,-1,-1,0,2009-08-23 18:02:38,How about we photoshop Bush's face on top of O...,2009,08,23,18,,photoshop bushs face top obama
1010527,1,:O,politics,1,1,0,2009-04-16 05:52:57,N.S.A. Intercepts Go Beyond Limits of Congress,2009,04,16,05,,nsa intercepts go beyond limits congress


In [15]:
comments[(comments["cleaned parent comment"]=="")]

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour,cleaned comment,cleaned parent comment
29,0,"This guy, there's no way he isn't trolling, ri...",RocketLeagueExchange,1,-1,-1,2016-11-06 03:39:29,Who?,2016,11,06,03,guy theres no way not trolling right,
42,0,1571049,counting,1,-1,-1,2016-12-17 06:56:28,"1,571,048",2016,12,17,06,,
95,0,"1,512,798",counting,1,-1,-1,2016-11-18 20:26:43,"1,512,797",2016,11,18,20,,
118,0,L,EnoughTrumpSpam,5,-1,-1,2016-11-08 21:06:01,I,2016,11,08,21,,
164,0,Hard to convey sarcasm over the Internet,Amd,1,1,0,2016-09-09 00:31:01,You're the only one,2016,09,09,00,hard convey sarcasm internet,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009559,0,At an [8]?,reddit.com,1,1,0,2010-11-27 07:24:08,**M A R I J U A N A**,2010,11,27,07,,
1010050,1,forgot to close my sarcasm tag,todayilearned,1,1,0,2010-08-10 16:43:46,Just now?,2010,08,10,16,forgot close sarcasm tag,
1010360,1,"Yes, I also thought that was the most importan...",atheism,10,10,0,2009-08-19 16:49:17,You're,2009,08,19,16,yes also thought important takeaway argument,
1010729,0,Indeed.,WeAreTheMusicMakers,1,1,0,2009-06-24 20:49:23,That she is.,2009,06,24,20,indeed,


In [16]:
comments = comments[(comments["cleaned comment"] != "")]
#comments = comments[(comments["cleaned parent comment"] != "")]

In [17]:
comments

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour,cleaned comment,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,10,16,23,nc nh,yeah get argument id prefer lived nc well
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,11,01,00,do know west teams play west teams east teams ...,blazers mavericks wests seed not even carry go...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,2016,09,22,21,underdogs earlier today since gronks announcem...,favored win
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,2016,10,18,21,meme not funny none new york nigga ones,deadass not kill buzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,2016,12,30,17,could use tools,yep confirm saw tool use made boy easportsmut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",2009,04,25,00,im sure iran korea technology create pigbirdhu...,no calling engineered pathogen reports virus b...
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,2009,05,14,22,whatever do not vote green,move typical recent donothing approach co emis...
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,2009,01,11,00,perhaps atheist conspiracy make christians loo...,screw disabledive got get church time
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,2009,01,23,21,slavs got country called kosovo,ive always unsettled hear lot jewish people sa...


In [18]:
# saving processed comments into a csv file
comments.to_csv("data/cleaned-train-balanced-sarcasm.csv", index = False)

In [19]:
# saving processed comments into a feather file
comments.reset_index().to_feather('data/cleaned-train-balanced-sarcasm.feather')

In [3]:
comments_feather = pd.read_feather(f'cleaned-train-balanced-sarcasm.feather').drop('index', axis=1)

In [4]:
comments_feather.head()

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,cleaned comment,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",nc nh,yeah get argument point id prefer lived nc well
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,know west teams play west teams east teams right,blazers mavericks wests seed not even carry go...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,underdogs earlier today since gronks announcem...,theyre favored win
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,meme funny none new york nigga ones,deadass kill buzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,could use one tools,yep confirm saw tool use made boy easportsmut


In [ ]:
#Simple preprocessing as baseline

In [20]:
comments = pd.read_csv('data/train-balanced-sarcasm.csv')

In [21]:
def simple_preprocessing(text):
     
    token = gensim.utils.simple_preprocess(text)
    final = ''
    accept_word_count = 0
    
    for word in range(len(token)):
        if (accept_word_count!=0):
            final += ' '
        final += token[word]
        accept_word_count += 1
            
    return final   
    

In [22]:
comments.head(10)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
5,0,"I don't pay attention to her, but as long as s...",only7inches,AskReddit,0,0,0,2016-09,2016-09-02 10:35:08,do you find ariana grande sexy ?
6,0,Trick or treating in general is just weird...,only7inches,AskReddit,1,-1,-1,2016-10,2016-10-23 21:43:03,What's your weird or unsettling Trick or Treat...
7,0,Blade Mastery+Masamune or GTFO!,P0k3rm4s7,FFBraveExvius,2,-1,-1,2016-10,2016-10-13 21:13:55,Probably Sephiroth. I refuse to taint his grea...
8,0,"You don't have to, you have a good build, buy ...",SoupToPots,pcmasterrace,1,-1,-1,2016-10,2016-10-27 19:11:06,What to upgrade? I have $500 to spend (mainly ...
9,0,I would love to see him at lolla.,chihawks,Lollapalooza,2,-1,-1,2016-11,2016-11-21 23:39:12,Probably count Kanye out Since the rest of his...


In [23]:
# removing the columns not really needed
comments.drop(['author', 'date'], axis = 1, inplace = True)
comments = comments.dropna()

In [24]:
# extraving the year, month, day and hour that the comment was made. May be useful in the future
comments['year'] = comments['created_utc'].map(lambda x: year_splitter(x))
comments['month'] = comments['created_utc'].map(lambda x: month_extractor(x))
comments['day'] = comments['created_utc'].map(lambda x: day_extractor(x))
comments['hour'] = comments['created_utc'].map(lambda x: hour_extractor(x))

In [25]:
gensim.utils.simple_preprocess("You do know west teams play against west teams")

['you', 'do', 'know', 'west', 'teams', 'play', 'against', 'west', 'teams']

In [26]:
simple_preprocessing("You do know west teams play against west teams")

'you do know west teams play against west teams'

In [27]:
comments['cleaned comment'] = comments['comment'].apply(simple_preprocessing)

comments['cleaned parent comment'] = comments['parent_comment'].apply(simple_preprocessing)

In [28]:
comments = comments[(comments["cleaned comment"] != "")]

In [29]:
comments

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour,cleaned comment,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,10,16,23,nc and nh,yeah get that argument at this point prefer is...
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,11,01,00,you do know west teams play against west teams...,the blazers and mavericks the wests and seed d...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,2016,09,22,21,they were underdogs earlier today but since gr...,they re favored to win
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,2016,10,18,21,this meme isn funny none of the new york nigga...,deadass don kill my buzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,2016,12,30,17,could use one of those tools,yep can confirm saw the tool they use for that...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",2009,04,25,00,sure that iran and korea have the technology t...,no one is calling this an engineered pathogen ...
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,2009,05,14,22,whatever you do don vote green,in move typical of their recent do nothing app...
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,2009,01,11,00,perhaps this is an atheist conspiracy to make ...,screw the disabled ve got to get to church on ...
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,2009,01,23,21,the slavs got their own country it is called k...,ve always been unsettled by that hear lot of j...


In [30]:
comments.to_csv("data/cleaned-train-balanced-sarcasm-baseline.csv", index = False)

In [31]:
comments


,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,year,month,day,hour,cleaned comment,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,10,16,23,nc and nh,yeah get that argument at this point prefer is...
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,11,01,00,you do know west teams play against west teams...,the blazers and mavericks the wests and seed d...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,2016,09,22,21,they were underdogs earlier today but since gr...,they re favored to win
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,2016,10,18,21,this meme isn funny none of the new york nigga...,deadass don kill my buzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,2016,12,30,17,could use one of those tools,yep can confirm saw the tool they use for that...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",2009,04,25,00,sure that iran and korea have the technology t...,no one is calling this an engineered pathogen ...
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,2009,05,14,22,whatever you do don vote green,in move typical of their recent do nothing app...
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,2009,01,11,00,perhaps this is an atheist conspiracy to make ...,screw the disabled ve got to get to church on ...
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,2009,01,23,21,the slavs got their own country it is called k...,ve always been unsettled by that hear lot of j...


In [33]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
corpus = [ 'This is the first document.',   'This document is the second document.','And this is the third one.','Is this the first document?']
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
X = vectorizer.fit_transform(corpus)


In [38]:
print(X)

  (0, 2)	0.38850983776953174
  (0, 6)	0.38850983776953174
  (0, 4)	0.3145321969754572
  (0, 8)	0.38850983776953174
  (0, 0)	0.3145321969754572
  (0, 1)	0.38850983776953174
  (0, 5)	0.25715068083178566
  (0, 3)	0.25715068083178566
  (0, 7)	0.25715068083178566
  (1, 4)	0.37782308125006125
  (1, 0)	0.7556461625001225
  (1, 5)	0.3088951258779955
  (1, 3)	0.3088951258779955
  (1, 7)	0.3088951258779955
  (2, 4)	0.46979138557992045
  (2, 8)	0.5802858236844359
  (2, 5)	0.38408524091481483
  (2, 3)	0.38408524091481483
  (2, 7)	0.38408524091481483
  (3, 2)	0.4485737628354793
  (3, 6)	0.4485737628354793
  (3, 0)	0.36315912086089214
  (3, 1)	0.4485737628354793
  (3, 5)	0.29690637740001563
  (3, 3)	0.29690637740001563
  (3, 7)	0.29690637740001563


In [36]:
print(vectorizer.get_feature_names())

['document', 'first', 'first document', 'is', 'is the', 'the', 'the first', 'this', 'this is']
